In [90]:
import numpy as np
import pandas as pd
from math import *

In [91]:
# n equal to number of features
n = 5

# m equal to number of samples
m = 100

In [92]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans

## Kmeans

In [93]:
class Clustering:
    def __init__(self, path='accounts_style.csv' ):
        self.path = path
        self.data = self.__preprecessing(path)
        self.data = self.__normalize_based_on_values([2, 2, 1.5, 1.25, 1.75])

    
    def getData(self):
        return self.data


    def fit_predict(self, n_clusters=6):
        kmeans = KMeans(n_clusters = n_clusters, init = 'k-means++', random_state = 42)
        self.kmeans = kmeans
        return kmeans.fit_predict(self.data)

    
    def cluster_centers(self):
        return self.kmeans.cluster_centers_


    def predict(self, input):
        return self.kmeans.predict(input)

    
    def __preprecessing(self, path):
        #select special part of the dataframe
        data = pd.read_csv(path)
        data = data.iloc[ : ,2:]

        #spilit data and add new culomns
        data[['Age1', 'Age2', 'Age3']] = data["Age"].str.split(",", n=2, expand=True)
        data[['Color1', 'Color2', 'Color3']] = data["Color"].str.split(",", n=2, expand=True)
        data[['Pattern1', 'Pattern2', 'Pattern3']] = data["Pattern"].str.split(",", n=2, expand=True)
        data[['Size1', 'Size2', 'Size3']] = data["Size"].str.split(",", n=2, expand=True)
        data[['Formal1', 'Formal2', 'Formal3']] = data["Formal"].str.split(",", n=2, expand=True)

        #drop unusage columns
        data.drop(columns =['Age', 'Color', 'Pattern', 'Size', 'Formal'], inplace = True)

        #convert type from str to float64
        data = data.astype('float64')

        #calculate the average of each row specialy
        data['Age_avg'] = self.__special_mean(data[['Age1', 'Age2', 'Age3']].values,20)
        data['Color-avg'] = self.__special_mean(data[['Color1', 'Color2', 'Color3']].values,20)
        data['Pattern_avg'] = self.__special_mean(data[['Pattern1', 'Pattern2', 'Pattern3']].values,20)
        data['Size-avg'] = self.__special_mean(data[['Size1', 'Size2', 'Size3']].values,20)
        data['Formal_avg'] = self.__special_mean(data[['Formal1', 'Formal2', 'Formal3']].values,20)

        #drop unusage columns
        data.drop(columns =['Age1', 'Age2', 'Age3', 'Color1', 'Color2', 'Color3', 'Pattern1',
         'Pattern2', 'Pattern3', 'Size1', 'Size2', 'Size3', 'Formal1', 'Formal2', 'Formal3'], inplace = True)

        return data



    def __normalize_based_on_values(self, values):
        #normalize data from 0 to 1
        normalized_data = pd.DataFrame(preprocessing.normalize(self.data), columns=self.data.columns)

        #put it in -1 to +1
        normalized_data = (normalized_data - 0.5) * 2

        #change the period for each component to change the influence
        normalized_data = normalized_data * values
        return normalized_data


    def __special_mean(self, selectedClothes, anomaly):
        # we have to ignore anomaly values in each component to have accessed the real mean value
        meanVector = []
        for i in range(np.shape(selectedClothes)[0]):
            componentSum = 0
            count = 0
            for j in range(np.shape(selectedClothes)[1]):
                if(selectedClothes[i][j] != anomaly):
                    componentSum += selectedClothes[i][j]
                    count += 1
            meanVector.append(componentSum/count)

        return meanVector




## recommend

In [94]:
class RecommendationSystem:
    
    def __init__(self, allClothes):
        self.allClothes = allClothes
        self.clustering = Clustering(path='accounts_style.csv')
        self.answers = self.clustering.fit_predict(n_clusters=6)
        self.preparedData = self.clustering.getData()
        self.preparedDataWithAnswers = self.preparedData
        self.preparedDataWithAnswers['Answers'] = self.answers

        
    def get_prepared_data(self):
        return self.preparedData

    def get_prepared_data_with_anwers(self):
        return self.preparedDataWithAnswers

    def recommend_based_on_questions(self, values, customer_feature1, customer_feature2, customer_feature3, customer_feature4, k1 = 4, k2=1, k3=1, k4=1):
    
        #k nearest neighbor
        k_indices1 = self.__one_recommend(customer_feature1, values, k1)
        k_indices2 = self.__one_recommend(customer_feature2, values, k2)
        k_indices3 = self.__one_recommend(customer_feature3, values, k3)
        k_indices4 = self.__one_recommend(customer_feature4, values, k4)
                
        # merge and unigue indices(union)
        k_indices = np.append(k_indices1, np.append(k_indices2, np.append(k_indices3, k_indices4)))
        
        # return the selected style indices 
        return k_indices


    def recommend_based_on_clothes(self, selectedClothes, values, anomaly = 20, k=5):
        
        #asume average of the each component for the as the customer vector of self.__oneRecommend little diffrent with the ordinary averages
        average_of_multi_tags = self.__special_mean(selectedClothes, anomaly)
        k_indeces = self.__one_recommend(average_of_multi_tags, values, k)
        return k_indeces


    def recommend_based_on_cluster(self, cluster_taste, k=30):
        
        return 1


    def __one_recommend(self, customer, values, k):
        # compute distances by one of the distance(distance_1, distance_2, ...) functions
        distances = np.array([self.__similarity2(customer, f, values) for f in self.allClothes])
        
        # return k nearest style indices
        return np.argsort(distances)[: k]



    def update_cluster_taste(self, prev_cluster_taste, comment, clothesInd, alpha = 0.1 ):
        #find the class number of clothes
        #clusterNumber = self.which_cluster(clothesInd)
        clusterNumber = 3
        
        #calculate the new replacement for selected cluster with weighted average
        weightedAverage = np.average([prev_cluster_taste[clusterNumber], comment], weights=[1,alpha])

        #update prev_Cluster_taste 
        prev_cluster_taste[clusterNumber] = weightedAverage
        return prev_cluster_taste


    def get_data(self):
        return self.preparedData

    def __clustring(self, clothes):
        return 1

        
    def which_cluster(ClothesInd):
        return 1


    def __similarity1(self, customer, clothes, values):
        lowestMines = self.__lowest_mines(customer, clothes)      
        return np.sum(values * np.abs(lowestMines))
    
    
    def __similarity2(self, customer, clothes, values):
        lowestMines = self.__lowest_mines(customer, clothes)      
        return np.sqrt(np.sum(values*((lowestMines) ** 2 )))


    def __lowest_mines(self, customer, clothes):
        bestClothes = []
        tmpArray1 = []
        for i in range(len(customer)):
            for j in range(np.shape(clothes)[1]):
                tmpArray1.append(abs((customer[i] - clothes[i][j])))
            bestClothes.append(min(tmpArray1))  
            tmpArray1 = []
        return np.array(bestClothes)

    def __special_mean(self, selectedClothes, anomaly):
        # we have to ignore anomaly values in each component to have access the real mean value
        meanVector = []
        for i in range(np.shape(selectedClothes)[0]):
            componentSum = 0
            count = 0
            for j in range(np.shape(selectedClothes)[1]):
                if(selectedClothes[i][j] != anomaly):
                    componentSum += selectedClothes[i][j]
                    count += 1
            meanVector.append(componentSum/count)

        return meanVector

            


    def extract_clothes(self, clothes):
        return 1


    def remove_duplicates(self, clothes):
        return 1
            

### 1.Test of recommend_based_on_questions function 
---
creating random data and check the answers of recommend_based_on_questions function, subfunctions.
its subfuctions including:
- __similarity2
- __lowest_mines
- __one_recommend



In [95]:
import random 

In [96]:
#create random clothes tags
my_list = [0, 1, 2, 3, 4]
clothes = np.zeros((m, n, 3))
for i in range(m):
    for j in range(n):
        for k in range(3):
            clothes[i, j, k] = random.choice(my_list)
    
print(clothes[:2])

[[[1. 1. 3.]
  [4. 0. 2.]
  [2. 3. 4.]
  [3. 2. 1.]
  [0. 2. 1.]]

 [[0. 3. 2.]
  [3. 1. 3.]
  [3. 0. 3.]
  [4. 2. 1.]
  [4. 1. 0.]]]


In [97]:
#create random personal tags
customer1 = np.array([random.choice(my_list) for j in range(n)])

#create 3 random selected style tags
customer2 = np.array([random.choice(my_list) for j in range(n)])
customer3 = np.array([random.choice(my_list) for j in range(n)])
customer4 = np.array([random.choice(my_list) for j in range(n)])
print(customer1, customer2, customer3, customer4)

[4 1 3 1 2] [4 3 1 1 2] [4 4 2 1 3] [1 4 4 2 3]


In [98]:
recommendationSystem = RecommendationSystem(clothes)

distances_indices = recommendationSystem.recommend_based_on_questions([1,1,1,1,1], customer1, customer2, customer3, customer4)

In [99]:
print(customer1)
print(clothes[distances_indices])
type(distances_indices)

[4 1 3 1 2]
[[[4. 3. 1.]
  [1. 1. 4.]
  [3. 4. 0.]
  [1. 3. 0.]
  [3. 3. 2.]]

 [[1. 4. 2.]
  [2. 1. 4.]
  [2. 3. 0.]
  [0. 2. 1.]
  [2. 0. 0.]]

 [[3. 4. 2.]
  [1. 1. 2.]
  [4. 4. 2.]
  [1. 0. 4.]
  [3. 2. 2.]]

 [[4. 3. 2.]
  [4. 2. 3.]
  [4. 3. 2.]
  [2. 1. 3.]
  [3. 3. 2.]]

 [[4. 1. 3.]
  [3. 2. 4.]
  [0. 2. 1.]
  [1. 4. 2.]
  [0. 0. 2.]]

 [[4. 3. 2.]
  [4. 2. 3.]
  [4. 3. 2.]
  [2. 1. 3.]
  [3. 3. 2.]]

 [[4. 1. 3.]
  [1. 4. 1.]
  [3. 0. 4.]
  [4. 4. 2.]
  [4. 3. 1.]]]


numpy.ndarray

### 2.Test of update_cluster_taste function 
---
creating random data and check the answers of update_cluster_taste function, subfunctions.
its subfuctions including:
- __which_cluster

In [100]:
#__wich_cluster always return 1 untill clustering implementation
clusterTaste = [5, 5, 5, 5, 5]
recommendationSystem.update_cluster_taste(prev_cluster_taste=clusterTaste, comment=1, clothesInd=145)

[5, 5, 5, 4.636363636363636, 5]

### 3.Test of recommend_based_on_clothes function 
---
creating random data and check the answers of recommend_based_on_clothes function, subfunctions.
its subfuctions including:
- __special_mean
- __one_recommend

In [101]:
clothesTest = [[1,2,10], [1,10,10], [3,4,10], [1,2,3], [2,3,4]]
indecesTest3 = recommendationSystem.recommend_based_on_clothes(clothesTest, values=[1,1,1,1,1], anomaly=10 )
clothes[indecesTest3]

array([[[0., 2., 1.],
        [1., 3., 3.],
        [1., 1., 4.],
        [0., 1., 2.],
        [1., 3., 4.]],

       [[4., 2., 0.],
        [3., 1., 4.],
        [3., 2., 4.],
        [0., 0., 2.],
        [0., 0., 3.]],

       [[2., 0., 3.],
        [1., 1., 3.],
        [1., 3., 4.],
        [3., 1., 2.],
        [0., 3., 2.]],

       [[1., 0., 0.],
        [1., 0., 2.],
        [3., 2., 4.],
        [2., 3., 0.],
        [3., 2., 3.]],

       [[3., 2., 2.],
        [1., 3., 2.],
        [3., 3., 2.],
        [0., 0., 2.],
        [2., 3., 3.]]])

In [104]:
preparedData = recommendationSystem.get_prepared_data_with_anwers()

In [105]:
preparedData

,Age_avg,Color-avg,Pattern_avg,Size-avg,Formal_avg,Answers
0,-0.351673,0.747211,-1.087918,-0.219796,-0.307714,4
1,-0.045289,0.443389,-1.133492,-0.333729,-0.039628,4
2,-0.845299,0.886751,-1.066987,-0.167468,-0.234456,0
3,-0.780011,1.049971,-1.042504,-0.487507,-0.148765,0
4,-0.718975,0.562050,-1.019616,-0.049039,-0.068654,4
...,...,...,...,...,...,...
395,-0.166603,-0.166603,-0.949981,-0.333302,0.495911,4
396,-0.592805,0.814390,-0.233524,-0.546402,-0.518704,5
397,-0.492443,0.713602,-0.143199,-0.307777,-0.958533,5
398,-0.158851,-0.158851,0.157034,-0.329425,-0.461196,2
